### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [4]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES: 
from fastbook import *                              # Getting all the Libraries. 
from fastai.callback.fp16 import *
from fastai.collab import *                         # Getting all the Libraries.
from fastai.tabular.all import *                    # Getting all the Libraries.

### **GETTING THE DATASET:**
- I will use the subset of [**MovieLens**](https://grouplens.org/datasets/movielens/) dataset. The entire dataset contains tens of millions of movie rankings i.e a combination of a movie ID, a user ID and a numeric rating. 

In [5]:
#@ GETTING THE DATASET: 
path = untar_data(URLs.ML_100k)                     # Getting Path to the Dataset. 
path.ls()                                           # Inspecting the Dataset. 

(#23) [Path('/root/.fastai/data/ml-100k/ua.test'),Path('/root/.fastai/data/ml-100k/u.item'),Path('/root/.fastai/data/ml-100k/u2.test'),Path('/root/.fastai/data/ml-100k/u.genre'),Path('/root/.fastai/data/ml-100k/u4.base'),Path('/root/.fastai/data/ml-100k/ub.base'),Path('/root/.fastai/data/ml-100k/u.info'),Path('/root/.fastai/data/ml-100k/u3.test'),Path('/root/.fastai/data/ml-100k/README'),Path('/root/.fastai/data/ml-100k/u1.test')...]

In [6]:
#@ READING THE DATASET: 
ratings = pd.read_csv(path/"u.data", delimiter="\t", header=None, 
                      names=["user", "movie", "rating", "timestamp"])   # Reading the Dataset. 
ratings.head()                                                          # Inspecting the Dataset. 

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
#@ INSPECTING THE COMBINATIONS: EXAMPLE: 
last_skywalker = np.array([0.98, 0.9, -0.9])                            # Initialization. 
user1 = np.array([0.9, 0.8, -0.6])                                      # Initialization. 
(user1*last_skywalker).sum()                                            # Getting Dot Product.  

2.1420000000000003

**Dot Product:**
- The mathematical operation of multiplying the elements of two vectors together and then summing up the result is called **Dot Product**. 

### **CREATING DATALOADERS:**

In [8]:
#@ READING THE DATASET: MOVIE TITLES: 
movies = pd.read_csv(path/"u.item", delimiter="|", encoding="latin-1", 
                     usecols=(0, 1), names=("movie", "title"), 
                     header=None)                                       # Reading the Dataset. 
movies.head()                                                           # Inspecting the Dataset. 

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [9]:
#@ MERGING THE DATAFRAMES: 
ratings = ratings.merge(movies)                                         # Merging the DataFrames. 
ratings.head()                                                          # Inspecting the DataFrame. 

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [10]:
#@ CREATING DATALOADERS: 
dls = CollabDataLoaders.from_df(ratings, item_name="title", bs=64)       # Initializing DataLoaders. 
dls.show_batch()                                                         # Inspecting the Batches. 

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [11]:
#@ CONVERTING INTO MATRICES: 
n_users = len(dls.classes["user"])                            # Getting Number of Users. 
n_movies = len(dls.classes["title"])                          # Getting Number of Movies. 
n_factors = 5                                                 # Initializing Latent Factors. 
user_factors = torch.randn(n_users, n_factors)                # Initializing User Factors. 
movie_factors = torch.randn(n_movies, n_factors)              # Initializing Movie Factors. 

In [14]:
#@ IMPLEMENTATION OF ONE HOT VECTORS: 
one_hot_3 = one_hot(3, n_users).float()                       # Initializing One Hot Vectors. 
user_factors.t() @ one_hot_3                                  # Initializing Dot Product.
user_factors[3]                                               # Inspecting One Hot Vectors.  

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

**Embedding:**
- The special layer that indexes into a vector using an integer but has its derivative calculated in such a way that it is identical to what it would have been if it had done a matrix multiplication with a one hot encoded vector is called **Embedding**. Multiplying by a one hot encoded matrix using the computational shortcut that it can be implemented by simply indexing directly. The thing that multiply the one hot encoded matrix is called the **Embedding Matrix**. 

### **COLLABORATIVE FILTERING**

In [16]:
#@ EXAMPLE OF A CLASS: 
class Example:                                 # Defining Class. 
    def __init__(self, a):                     # Constructor Function. 
        self.a = a                             # Initialization. 
    def say(self, x):                          # Defining Function. 
        return f"Hello {self.a}, {x}"
#@ IMPLEMENTATION OF CLASS: 
ex = Example("Thinam")                         # Initialization. 
ex.say("how are you?")                         # Implementation. 

'Hello Thinam, how are you?'

In [17]:
#@ EXAMPLE OF A CLASS WITH PYTORCH MODULE: 
class DotProduct(Module):                                           # Defining Class. 
    def __init__(self, n_users, n_movies, n_factors):               # Constructor Function. 
        self.user_factors = Embedding(n_users, n_factors)           # Initialization. 
        self.movie_factors = Embedding(n_movies, n_factors)         # initialization. 
    
    def forward(self, x):                                           # Forward Propagation Function. 
        users = self.user_factors(x[:, 0])                          # Implementation. 
        movies = self.movie_factors(x[:, 1])                        # Implementation. 
        return (users * movies).sum(dim=1)                          # Matrix Multiplication. 

**Note:**
- The input of the model is a tensor of shape batch_size X 2 where the first column contains the user IDs and the second column contains the movie IDs. 

In [20]:
#@ GETTING A BATCH:
x, y = dls.one_batch()                                 # Getting a Batch. 
x.shape                                                # Inspecting the Shape. 

torch.Size([64, 2])

In [21]:
#@ TRAINING SIMPLE MODEL FROM SCRATCH: 
model = DotProduct(n_users, n_movies, 50)              # Initializing the Class. 
learn = Learner(dls, model, loss_func=MSELossFlat())   # Initializing the Learner. 
learn.fit_one_cycle(5, 5e-3)                           # Training the Model. 

epoch,train_loss,valid_loss,time
0,1.346760,1.296290,00:07
1,1.054427,1.091131,00:07
2,0.954217,0.992386,00:07
3,0.845169,0.897380,00:06
4,0.791750,0.882070,00:07


In [22]:
#@ DEFINING THE DOT PRODUCT CLASS WITH RANGE: 
class DotProduct(Module):                                               # Defining Class.
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)): # Constructor Function. 
        self.user_factors = Embedding(n_users, n_factors)               # Initialization.
        self.movie_factors = Embedding(n_movies, n_factors)             # Initialization. 
        self.y_range = y_range                                          # Initialization. 

    def forward(self, x):                                               # Forward Propagation Function. 
        users = self.user_factors(x[:, 0])                              # Implementation. 
        movies = self.movie_factors(x[:, 1])                            # Implementation. 
        return sigmoid_range((users*movies).sum(dim=1), *self.y_range)  # Matrix Multiplication and Range. 

#@ TRAINING SIMPLE MODEL FROM SCRATCH: 
model = DotProduct(n_users, n_movies, 50)                               # Initializing the Class. 
learn = Learner(dls, model, loss_func=MSELossFlat())                    # Initializing the Learner. 
learn.fit_one_cycle(5, 5e-3)                                            # Training the Model. 

epoch,train_loss,valid_loss,time
0,0.967142,0.992117,00:07
1,0.875934,0.914618,00:07
2,0.681316,0.871721,00:07
3,0.484174,0.873949,00:07
4,0.376960,0.877157,00:07
